In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.vision import *
from fastai.callbacks.hooks import *
from fastai.utils.mem import *
from PIL import Image
from numpy import array

import cv2
import numpy as np

from torchvision.utils import save_image
import torch
import matplotlib

In [3]:
def segs2onehot(frame, nchannels):
    """
    Converts a tensor of shape (1, imgx, imgy)
    with discrete values in the range(nchannels)
    to a onehot style tensor of shape (nchannels, imgx, imgy)
    """
    *_, nx, ny = frame.shape
    x,y = np.meshgrid(np.arange(nx), np.arange(ny))
    res = np.zeros((nchannels, nx, ny))
    res[frame.T.ravel(), x.ravel(), y.ravel()] = 1
    return res

## Import Semented Image

In [4]:
seg_im = cv2.imread('segImage1.png')

In [5]:
seg_im.shape

(360, 480, 3)

## Get one channel from seg_im

In [6]:
seg = seg_im[:, :, 0]; seg.shape

(360, 480)

## Perform one-hot encoding on seg

In [7]:
segOH = segs2onehot(seg, 32)

In [8]:
segOH.shape

(32, 360, 480)

## Import Optical flow matrices, single out the first matrix

In [9]:
OF_matrices = np.load('optFlowMat1.npy')
optflow1 = OF_matrices[0]

In [10]:
optflow1.shape

(2, 360, 480)

In [11]:
def assignFlowToSegs(segs, flow, nchannels):
    """
    Maps a tensor flow of shape (nx, ny, nv)
    using a tensor segs of shape (1, nx, ny)
    with discrete values in the range(nchannels)
    onto a onehot style tensor of shape (nchannels, nv, nx, ny)
    """
    nx, ny, nv = flow.shape
    x,y = np.meshgrid(np.arange(nx), np.arange(ny))
    res = np.zeros((nchannels, nv, nx, ny))
    res[segs.T.ravel(), :, x.ravel(), y.ravel()] = flow[x.ravel(), y.ravel(), :]
    return res

## Assign optflow to segmented image (segOH)

In [12]:
vector = assignFlowToSegs(segOH, optflow1, 32)

IndexError: arrays used as indices must be of integer (or boolean) type

In [13]:
def assignFlowToSegs(segs, flow, nchannels):
    """
    Maps a tensor flow of shape (nx, ny, nv)
    using a tensor segs of shape (1, nx, ny)
    with discrete values in the range(nchannels)
    onto a onehot style tensor of shape (nchannels, nv, nx, ny)
    """
    nx, ny, nv = flow.shape
    x,y = np.meshgrid(np.arange(nx), np.arange(ny))
    res = np.zeros((nchannels, nv, nx, ny))
    res[segs.T.ravel(), :, x.ravel(), y.ravel()] = flow[x.ravel(), y.ravel(), :]
    return res

In [14]:
nx, ny, nv = optflow1.shape

In [15]:
print('nx', nx)
print('ny', ny)
print('nv', nv)

nx 2
ny 360
nv 480


In [16]:
x,y = np.meshgrid(np.arange(nx), np.arange(ny))

In [17]:
x.shape

(360, 2)

In [18]:
y.shape

(360, 2)

In [19]:
nchannels = 32

In [20]:
res = np.zeros((nchannels, nv, nx, ny))

In [21]:
res.shape

(32, 480, 2, 360)

In [22]:
res[segOH.T.ravel(), :, x.ravel(), y.ravel()] = optflow1[x.ravel(), y.ravel(), :]

IndexError: arrays used as indices must be of integer (or boolean) type

In [39]:
cpr = np.dot(segOH, optflow1)

ValueError: shapes (32,360,480) and (2,360,480) not aligned: 480 (dim 2) != 360 (dim 1)

In [41]:
segOH.shape

(32, 360, 480)

In [42]:
optflow1.shape

(2, 360, 480)

In [43]:
seg1 = segOH[None, :,:,:]
# 01 x 02 x 360 x 480
# 32 x 01 x 360 x 480
seg1.shape

(1, 32, 360, 480)

In [44]:
opt1 = optflow1[:, None, :, :]

In [45]:
opt1.shape

(2, 1, 360, 480)

In [73]:
??np.dot

In [58]:
cc = (opt1 * seg1)

In [59]:
cc.shape

(2, 32, 360, 480)

In [77]:
c2 = cc.T;
c2.shape

(480, 360, 32, 2)

In [79]:
# 360x480x2x32
c2.T;
c2.shape

(480, 360, 32, 2)

In [80]:
vector = torch.from_numpy(c2)

In [81]:
vector.shape

torch.Size([480, 360, 32, 2])

In [82]:
vector = torch.transpose(vector, 0, 1)
vector = torch.transpose(vector, 2, 3)
vector.shape

torch.Size([360, 480, 2, 32])

## Average Pooling

In [85]:
pooled = torch.nn.AvgPool3d(kernel_size=3, stride=3, padding=0)

In [88]:
??torch.nn.

In [92]:
len(cc.shape)

4